In [30]:
import pandas as pd
import numpy as np
import torch
import os
import umap

from tqdm import tqdm

In [31]:
DATA_FOLDER = "./data/"
CHECKPOINT_ID = "TEST"
DIR = os.path.join(DATA_FOLDER, CHECKPOINT_ID)

In [32]:
# UMAP settings:

N_NEIGHBORS = 3
MIN_DIST = 0.1
METRIC = 'cosine'
SEMI_SUPERVISED = False
SEMI_SUPERVISED_N = 10

In [33]:
def get_top_n_indices(df, column_name, n):
    return np.argsort(df[column_name].values)[n:]

In [41]:
embeddings = np.load(os.path.join(DIR, "y_embeddings.npy"))
metadata = pd.read_csv(os.path.join(DIR, "metadata.csv"), index_col=0)

In [42]:
embeddings

array([[0.26739013, 0.287149  , 0.3533957 , ..., 0.9722748 , 0.42294902,
        0.4622926 ],
       [0.5191779 , 0.00640219, 0.37415892, ..., 0.4497561 , 0.8127194 ,
        0.22906065],
       [0.41835755, 0.8100909 , 0.9690676 , ..., 0.13887334, 0.7410607 ,
        0.8147367 ],
       ...,
       [0.14864844, 0.9890428 , 0.6018168 , ..., 0.65867513, 0.94109213,
        0.2263087 ],
       [0.10550368, 0.1704374 , 0.60778856, ..., 0.61049473, 0.9952317 ,
        0.08921719],
       [0.8005367 , 0.97020966, 0.8839034 , ..., 0.46284622, 0.14257395,
        0.47934908]], dtype=float32)

In [43]:
metadata

,audio_file,x_b,x_d,y_hat_b,y_hat_d,brightness_diff,depth_diff,p_drive,p_hpf,p_bias,p_output_db,p_hat_drive,p_hat_hpf,p_hat_bias,p_hat_output_db
id,,,,,,,,,,,,,,,
0,./audio0.wav,0.491584,0.366129,0.179557,0.043799,-0.312027,-0.322329,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
1,./audio1.wav,0.491584,0.366129,0.471837,0.252225,-0.019747,-0.113904,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
2,./audio2.wav,0.491584,0.366129,0.329368,0.900148,-0.162216,0.534020,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
3,./audio3.wav,0.491584,0.366129,0.205323,0.487679,-0.286260,0.121551,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
4,./audio4.wav,0.491584,0.366129,0.626251,0.029368,0.134667,-0.336761,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,./audio995.wav,0.491584,0.366129,0.798466,0.889531,0.306882,0.523403,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
996,./audio996.wav,0.491584,0.366129,0.763310,0.865694,0.271727,0.499565,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
997,./audio997.wav,0.491584,0.366129,0.219154,0.087598,-0.272429,-0.278530,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0


In [48]:
top_bright = get_top_n_indices(metadata, 'brightness_diff', 10)

In [49]:
metadata.iloc[top_bright]

,audio_file,x_b,x_d,y_hat_b,y_hat_d,brightness_diff,depth_diff,p_drive,p_hpf,p_bias,p_output_db,p_hat_drive,p_hat_hpf,p_hat_bias,p_hat_output_db
id,,,,,,,,,,,,,,,
360,./audio360.wav,0.491584,0.366129,0.000655,0.500069,-0.490929,0.133940,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
399,./audio399.wav,0.491584,0.366129,0.002662,0.144427,-0.488921,-0.221701,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
123,./audio123.wav,0.491584,0.366129,0.003141,0.389392,-0.488442,0.023263,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
701,./audio701.wav,0.491584,0.366129,0.003706,0.023145,-0.487878,-0.342983,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
720,./audio720.wav,0.491584,0.366129,0.003834,0.247936,-0.487750,-0.118192,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
638,./audio638.wav,0.491584,0.366129,0.004523,0.715867,-0.487060,0.349738,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
910,./audio910.wav,0.491584,0.366129,0.005135,0.245827,-0.486449,-0.120302,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
334,./audio334.wav,0.491584,0.366129,0.005464,0.106091,-0.486120,-0.260038,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
835,./audio835.wav,0.491584,0.366129,0.006024,0.164321,-0.485560,-0.201808,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0
